In [20]:
# zabiralnazi@yahoo.com

In [21]:
# cnn benchmark

In [1]:
from scipy import io, signal
import matplotlib.pyplot as plt
import dtcwt
import numpy as np
import itertools
import pywt

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# Load DWT-denoised time-series data
SubA = io.loadmat('SubA_5chan_3LRF.mat')
X = SubA['EEGDATA']
y = SubA['LABELS']
print(X.shape)
print(y.shape)

(5, 1024, 270)
(270, 1)


In [12]:
X = np.swapaxes(X,0,2)
print(X.shape)

(270, 1024, 5)


In [16]:
y[y==1] = 0
y[y==2] = 1
y[y==3] = 2

In [17]:
from keras.utils import to_categorical

y = to_categorical(y,3)

In [19]:
print(y[0:5])

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [23]:
from keras.models import Sequential
from keras.layers import Reshape, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dropout, Dense

In [27]:
temporal_dimension = X.shape[1]
num_channels = X.shape[2]
num_classes = 3

In [31]:
input_shape = temporal_dimension

In [41]:
model = Sequential()
model.add(Conv1D(100, 10, activation='relu', input_shape=(temporal_dimension, num_channels)))
model.add(Conv1D(100, 10, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(160, 10, activation='relu'))
model.add(Conv1D(160, 10, activation='relu'))
model.add(GlobalAveragePooling1D())
#model.add(Dense(100, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(num_classes, activation='sigmoid'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_21 (Conv1D)           (None, 1015, 100)         5100      
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 1006, 100)         100100    
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 335, 100)          0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 326, 160)          160160    
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 317, 160)          256160    
_________________________________________________________________
global_average_pooling1d_6 ( (None, 160)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 160)               0         
__________

In [42]:
model.compile(loss='binary_crossentropy',
                optimizer='adam', metrics=['accuracy'])

BATCH_SIZE = 128
EPOCHS = 5

history = model.fit(X,
                      y,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      validation_split=0.1,
                      verbose=1)

Train on 243 samples, validate on 27 samples
Epoch 1/5
243/243 [==============================] - 10s 41ms/step - loss: 0.6929 - acc: 0.5391 - val_loss: 0.6910 - val_acc: 0.6667
Epoch 2/5
243/243 [==============================] - 9s 37ms/step - loss: 0.6900 - acc: 0.6667 - val_loss: 0.6839 - val_acc: 0.6667
Epoch 3/5
243/243 [==============================] - 11s 45ms/step - loss: 0.6811 - acc: 0.6667 - val_loss: 0.6665 - val_acc: 0.6667
Epoch 4/5
243/243 [==============================] - 11s 44ms/step - loss: 0.6620 - acc: 0.6667 - val_loss: 0.6389 - val_acc: 0.6667
Epoch 5/5
243/243 [==============================] - 11s 46ms/step - loss: 0.6409 - acc: 0.6667 - val_loss: 0.6481 - val_acc: 0.6667


In [25]:
# reference

In [24]:
# https://blog.goodaudience.com/introduction-to-1d-convolutional-neural-networks-in-keras-for-time-sequences-3a7ff801a2cf